# AutoETS Model

In [ ]:
from typing import List
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

from statsforecast.models import AutoETS
from statsforecast.core import StatsForecast
from utilsforecast.evaluation import evaluate
from src.data import load_full_data, process_wide_df
from utilsforecast.plotting import plot_series

In [ ]:
# Use competition metric to evaluate our cross-validation predictions.
def CompetitionMetric(
    df: pd.DataFrame,
    models: List[str],
    id_col: str = "unique_id",
    target_col: str = "y",
) -> pd.DataFrame:
    """Computes the competition metric"""
    error = df[models].sub(df[target_col], axis=0)

    sum_abs_error = error[models].abs().sum(axis=0)
    abs_sum_error = error[models].sum(axis=0).abs()
    score = sum_abs_error + abs_sum_error
    score /= df[target_col].sum()

    score.index.name = id_col
    score = score.reset_index()
    return score

In [ ]:
# Model inputs
horizon = 13
freq = "W-MON"

sf = StatsForecast(models=[AutoETS(model='ZZZ'),],
                   freq="7d",
                   n_jobs=-1,
                   verbose=True)

In [ ]:
%%time

df = process_wide_df(load_full_data())

# Cross-validation run
yhat_cv = sf.cross_validation(df=df,
                              n_windows=1,
                              h=horizon,
                              step_size=13,
                              refit=True)

In [ ]:
yhat_cv = yhat_cv.to_pandas()

# Enforce non-negativity
yhat_cv[str(sf.models[0])] = yhat_cv[str(sf.models[0])].clip(0)

# Evaluate the cross-validation set on the competition metric
cv_metric = evaluate(
    yhat_cv.drop(columns=['cutoff']),
    metrics=[CompetitionMetric],
)
print(f"{cv_metric}")

In [ ]:
%%time

# Fit the model
sf.fit(df = df)

# Generate test predictions
yhat_test = sf.predict(h=horizon)
yhat_test = yhat_test.to_pandas()

# Enforce non-negativity
yhat_test[str(sf.models[0])] = yhat_test[str(sf.models[0])].clip(0)

In [ ]:
import matplotlib.pyplot as plt

f, ax = plt.subplots(1, 1)
df.to_pandas().groupby("ds")[["y"]].sum().plot(ax=ax)
yhat_test.groupby("ds")[["AutoETS"]].sum().plot(ax=ax)
plt.show()